<a href="https://colab.research.google.com/github/sahaanirbannew/spacy-custom-NER-bird-names/blob/main/Program_fetch_tweet_based_on_search_term%2C_create_training_data%2C_ML_model_to_NER_bird_names.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports & Function definitions

In [15]:
def what_search_term():
  search_term = "indiAves"        ## CHANGE THIS. 
  return search_term
  

import tweepy
import time
import pickle
import csv
import datetime 
from datetime import datetime 
import os, sys
from google.colab import drive
import pandas as pd 
import re
from spacy.tokens import DocBin
from tqdm import tqdm
from spacy.util import filter_spans
import shutil
from zipfile import ZipFile
import tensorflow as tf

def connect_to_google_drive():
  drive.mount("/content/drive", force_remount=False)

def extract_model_best_from_archive():
  file_name = "/content/model-best.zip"
  with ZipFile(file_name, 'r') as zip:
    zip.extractall("/content/model-best")

def is_GPU_on():
  if tf.test.gpu_device_name() == "": return False
  else: return True 

def create_twitter_app_obj():
  consumer_key = "iPaIdR8GRI59yTJMs0Es0dIBN"
  consumer_secret = "pLadg3UaLeK3yKDujRMChRN3p8hUDBOjBsuOBy8j8ERr4zz1vs"
  access_token = "39085479-AabHt6bmFSbClDfUZuHjModYPAxVlOxHeMA79UyVt"
  access_token_secret = "3IqXDISfqg14wzMNNn2AX4KYG9Wfkltt21QxKasE4YNnG"
  # Creating the authentication object
  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  # Setting your access token and secret
  auth.set_access_token(access_token, access_token_secret)
  # Creating the API object while passing in auth information

  try:
    api = tweepy.API(auth) 
    return api
  except:
    print("Error: Error making the Twitter Object.")
    return 0

def get_google_drive_folder_path():
  path = "/content/drive/My Drive/IndiAves/" 
  return path 

def get_since_id(path,search_word):
  file = open(path+"since_id",'rb')
  try:
    since_id_data = pickle.load(file)
  except Exception as e:
    print(str(e))
    return 0
  try: 
    since_id = since_id_data[search_word]
    return since_id
  except:
    print("Info: This looks like a new search.")
    return 0

def load_all_birds_list(path):
  file = open(path+"bird_list_df",'rb')
  bird_list_df = pickle.load(file)
  try: 
    return bird_list_df
  except:
    print("Error: No bird list found.")
    return 0
  
def load_training_data(path):
  file = open(path+"training_data",'rb')
  training_data = pickle.load(file)
  try: 
    return training_data
  except:
    print("Error: No training data found.")
    return 0

def load_nlp_model_():
  import spacy
  #nlp = spacy.load("en_core_web_sm")
  nlp = spacy.blank("en")
  print(nlp.pipe_names)
  return nlp

def create_doc_bin(training_data,nlp):
  from spacy.tokens import DocBin
  from tqdm import tqdm
  from spacy.util import filter_spans
  doc_bin = DocBin()
  for training_example  in tqdm(training_data['annotations']): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        #if span is None:
        #    print("Skipping entity")
        #else:
        #    ents.append(span)
        if span is not None:
          ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)
  doc_bin.to_disk("training_data.spacy") # save the docbin object

def export_training_data(training_data,path):
  filepath = open(path+"training_data",'wb') 
  pickle.dump(training_data, filepath)                     
  filepath.close()
  print(len(training_data), "examples of training data saved.") 

def initialisation():
  !pip install tweet-preprocessor
  !python -m spacy download en_core_web_sm
  !pip install spacy[transformers]

def import_base_config_file(google_drive_folder_path):
  shutil.copyfile(google_drive_folder_path+"base_config.cfg", "/content/base_config.cfg")

def basic_preprocess(tweet):
  import preprocessor as p
  p.set_options(p.OPT.EMOJI, p.OPT.MENTION, p.OPT.URL, p.OPT.SMILEY, p.OPT.NUMBER,p.OPT.HASHTAG)
  tweet = tweet.lower()
  tweet = tweet.replace("\n"," ")  
  tweet = tweet.replace("\\n"," ")
  tweet = tweet.replace("'","")
  tweet = tweet[1:] 
  tweet = p.clean(tweet)
  tweet = re.sub(r'[^\w\s]', ' ', tweet)
  tweet = re.sub(r' x..', '', tweet)
  tweet = re.sub(r' +', ' ', tweet) #' +', ' '
  #tweet = re.sub(r' n. ', '', tweet) 
  tweet = tweet.replace("x9c","")
  tweet = tweet.strip()
  return tweet

def preprocess_tweets_arr_batch(tweets):
  new_tweets_ = []
  for tweet in tweets:
    tweet = basic_preprocess(tweet)
    if len(tweet) > 1:
      new_tweets_.append(tweet)
  return new_tweets_

def add_to_database(api, search_word, ext_path, since_id):
  new_search = "#" +search_word + " -filter:retweets" 
  count = 0
  csvFile = open(ext_path+search_word+'.csv', 'a')
  csvFile2 = open('/content/temp_.csv', 'w') 

  csvWriter = csv.writer(csvFile)
  csvWriter2 = csv.writer(csvFile2)

  for tweet in tweepy.Cursor(api.search,q=new_search,count=100,
                            lang="en",
                            since_id=since_id, tweet_mode="extended").items(): 
    if tweet.id > since_id:
      since_id = tweet.id
    
    media_url=""
    if tweet.entities.get('media', []): media_url = tweet.entities.get('media', [])[0]['media_url']

    hashtags = []
    for tag in tweet.entities["hashtags"]:
      hashtags.append(tag["text"]) 
    
    csvWriter.writerow([tweet.created_at, tweet.id, tweet.user.screen_name.encode('utf-8'), tweet.user.location.encode('utf-8'), tweet.full_text.encode('utf-8'), media_url, hashtags])
    csvWriter2.writerow([tweet.created_at, tweet.id, tweet.user.screen_name.encode('utf-8'), tweet.user.location.encode('utf-8'), tweet.full_text.encode('utf-8'), media_url, hashtags])
    count +=1
  print(datetime.now(),count, "tweets retrieved.")
  return since_id 

def update_since_id(since_id, path,search_word):
  filepath = open(path+"since_id",'rb')
  since_id_data = pickle.load(filepath)
  since_id_data[search_word]=since_id
  filepath = open(path+"since_id",'wb') 
  pickle.dump(since_id_data, filepath)                     
  filepath.close()

def train_model():
  !python -m spacy init fill-config base_config.cfg config.cfg
  !python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy --gpu-id 0

def import_best_model(google_drive_folder_path):
  shutil.copyfile(google_drive_folder_path+"model-best.zip", "/content/model-best.zip")

def archive_best_model_in_google_drive(google_folder_path):
  shutil.make_archive(google_folder_path+"model-best", 'zip', "/content/model-best")


In [2]:
def update_training_data(training_data,new_tweets,all_birds_name):
  
  for tweet in new_tweets:
    tweet_bird_found = False 

    for bird in all_birds_name:
      if tweet.find(bird.strip())>-1:
        print(tweet)
        print(bird, tweet.find(bird), tweet.find(bird)+len(bird))
        training_data["annotations"].append({'text': tweet, 'entities': [(tweet.find(bird), tweet.find(bird)+len(bird), 'BIRDNAME')]})
        tweet_bird_found = True  
        print("---")
    
    if tweet_bird_found == False :
        doc = nlp_ner(tweet)
        print(tweet)
        suggestion = str(doc.ents).replace("(","").replace(")","").replace(",","")
        print("Suggestion: "+suggestion)
        birds_input = input("Birds,:").split(",")
        for bird_ in birds_input: 
          bird_ = bird_.strip()
          print(bird_, tweet.find(bird_),tweet.find(bird_)+len(bird_))
          training_data["annotations"].append({'text': tweet, 'entities': [(tweet.find(bird_), tweet.find(bird_)+len(bird_), 'BIRDNAME')]})
          print("---")
  return training_data 

# Main Program - 1

Run this only once.

Note:


*   Make sure you've chosen to use GPU.
*   List item



In [3]:
initialisation() 
import spacy 

#what is the search term?
search_word = what_search_term() 
print(search_word)

#connect to google drive
connect_to_google_drive()

#connect to twitter api. 
twitter_api_obj = create_twitter_app_obj() 

#get Google Drive folder path
google_drive_folder_path = get_google_drive_folder_path() 
print(google_drive_folder_path)

#get since_id based on the search term
since_id = get_since_id(google_drive_folder_path,search_word) 
#print(since_id)

#load all birds list 
all_birds = load_all_birds_list(google_drive_folder_path)

#load training data
training_data = load_training_data(google_drive_folder_path) 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-08-31 04:55:45.871300: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 5.0 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.6 MB/s 
     |████████████████████████████████| 1.1 MB 10.2 MB/s 
     |████████████████████████████████| 4.7 MB 44.4 MB/s 
     |████████████████████████████████| 120 kB 59.3 MB/s 
     |████████████████████████████████| 6.6 MB 43.2 MB/s 
indiAves
Mounted at /content/drive
/content/drive/My Drive/IndiAves/


In [16]:
#Set up Model Training. 
# PS: GPU needed. 
nlp = load_nlp_model_() #this is spacy stuff. 

is_GPU_on = is_GPU_on() 
print("is_GPU_on",is_GPU_on)

if is_GPU_on == True: 
  create_doc_bin(training_data,nlp)

  #train the model 
  import_base_config_file(google_drive_folder_path)

  #trains the model.
  train_model() 

  #archive the best model to Google Drive. 
  archive_best_model_in_google_drive(google_drive_folder_path) 

if is_GPU_on == False: 
  import_best_model(google_drive_folder_path)
  extract_model_best_from_archive()

#load the best model
nlp_ner = spacy.load("model-best")

[]
is_GPU_on False


# Main Program - 2

Keep re-running this!

In [ ]:
#fetch tweets
since_id = add_to_database(twitter_api_obj,search_word, google_drive_folder_path, since_id) 
new_tweets_df = pd.read_csv("/content/temp_.csv", names=["created_at", "tweet_id", "user", "location", "tweet", "media_url", "hashtags"])
new_tweets = preprocess_tweets_arr_batch(new_tweets_df["tweet"].tolist())
all_birds_name = all_birds["bird_name"].tolist() 

#Update training data. ###NEEDS USER INTERVENTION. 
update_training_data(training_data,new_tweets,all_birds_name)
backup_training_data = training_data 

#Exports training data 
export_training_data(training_data,google_drive_folder_path)

#Update since_id
update_since_id(since_id,google_drive_folder_path,search_word)

#delete temp_.csv 
os.remove("/content/temp_.csv")

In [ ]:
new_tweets

['belted kingfisher rocks a pompadour',
 'enjoying a sunset survey with their s',
 'weekday sunset',
 'ruffled feathers',
 'for me its how much water gulls get through theyre about the only bird were getting regularly at the moment and they drink a remarkable amount of water sometimes they only come for water and ignore the food',
 'on fort island trail in florida',
 'crestfallen rook some birds just take on a character of their own as the ink goes on i wonder what hes done to look so disappointed',
 'birds of the day',
 'a giant kingfisher in bullet mode photographed in',
 'pearly eyed thrasher hes always on the lighthouse and very used to people i took this shots very close he still looking for food ignoring me',
 'sa towns travel beaches nature',
 'bird photo dl on photoac free adobe stock pixta',
 'sanderling today at rossall point',
 'stunning',
 'a red breasted nuthatch',
 'dunlin today at rossall point',
 'short evening visit to on day since fledgling and the young was in their 

# Stand alone test.

In [17]:
sentence = "this is a preprocessed text and it should find spotted dove"
doc = nlp_ner(sentence)
print(doc.ents)

(spotted dove,)
